## <span style="color:blue">PL/SQL </span> 

In this section, we introduce PL/pgSQL on postgreSQL. For Oracle developpers, the syntax is very similar.

Before begining this notebook, make sure to etablish a connexion before going on :

In [27]:
-- connection: host='localhost' dbname='ds2' user='ds2' 


 ## <span style="color:blue">Let's start </span>

A PL/SQL program is consisted of 4 sections :

DECLARE (optional)
Variable definition 

BEGIN (mandatory)
Implementing the business logic and have to contain at least one declaration SQL or PL/SQL

EXCEPTION (optional)
Exception management 

END; ( mandatory ) 







## <span style="color:blue">Enabling Output of a PL/SQL Block</span> 

To enable output in postgreSQL, execute the following command before running the PL/SQL block


In [28]:
set client_min_messages = LOG;

You can filter the trace level to DEBUG5, DEBUG4, DEBUG3, DEBUG2, DEBUG1, LOG, NOTICE, WARNING, ERROR, FATAL, and PANIC.

Ready for your first procedure :

In [29]:
DO $$
BEGIN
raise log 'Hello';
END;
$$

LOG:  Hello


## <span style="color:blue">Your first Program</span>

PL/SQL is a language which may be used in some specifics database objects : 
* Triggers
* Stored procedures
* Functions
* Anonymous block

In PL/SQL, you can run a PL/SQL code without creating a database object like stored procedure or a trigger, it's what we call an Anonymous block. In postgres, to run an anonymous block you can do it with the DO command :


In [30]:

DO $$                      -- DO (mandatory) indicates to Postgres it's an anonymous block.
<< label >>                -- you can define a label we will see the interest  later
DECLARE                    -- Optional section for declaring variables
BEGIN                     -- computing block (mandatory) : have to contain at least one instruction
   Raise notice 'Hello';   -- Print Hello
END                        -- computing block end (mandatory)
$$    

NOTICE:  Hello


Note that the double dollar ($$) is a substitution of a single quote (‘).

Bravo, you ran your first PL/SQL program.

Here, we create the same program with via a function :

In [31]:
DROP function if exists f_hello( text);

CREATE or replace FUNCTION f_hello(v_myTxt text) RETURNS text 
AS $BODY$
DECLARE
  v_hello text ='Hello';
BEGIN
  RETURN v_hello||' '||v_myTxt;
END
$BODY$
LANGUAGE plpgsql; 


Now, we can call our function :

In [32]:
select f_hello('Guy');

1 row(s) returned.


f_hello
Hello Guy


We can do the same thing through a stored procedure :

In [33]:
DROP procedure if exists p_hello( varchar(50));

CREATE PROCEDURE p_hello(v_myTxt varchar(50)) 
LANGUAGE plpgsql 
AS $BODY$ 
DECLARE
   v_hello text ='Hello';
BEGIN
   Raise notice '% : % ', v_hello, v_myTxt;
END
$BODY$;

In [34]:
call p_hello('Guy');


NOTICE:  Hello : Guy 


Now you are ready to write your first PL/SQL code.

# <span style="color:blue">Scalar variables</span> 

If you are famillar with programming it's very similar, you declare the name, the datatype and evantually the default value.

The datatype list supported in postgres is available here :
https://www.postgresql.org/docs/11/datatype.html

Here is a simple code displaying a variable :

In [35]:
DO $$                      
DECLARE
costype varchar(50) default 'Galaxy';
BEGIN                     
   Raise info 'var costype : %',costype; 
END  
$$ 
LANGUAGE plpgsql;

INFO:  var costype : Galaxy


You can define the type of a variable from : 
* an existing colum  with the syntax %TYPE
* an existing local variable 

In [36]:
DO $$                      
DECLARE
v_myTitle products.title%TYPE;
v_copyMyTitleType v_myTitle%TYPE;
BEGIN  
   v_myTitle := 'Coucou';
   Raise info 'v_myTitle : %',v_myTitle; 
   v_copyMyTitleType := length(v_myTitle)::text; -- we need to cast data from int to text
   Raise info 'v_copyMyIntType : %',v_copyMyTitleType; 
END  
$$ 
LANGUAGE plpgsql;

INFO:  v_myTitle : Coucou
INFO:  v_copyMyIntType : 6


# <span style="color:blue">Composite variables</span> 

Now, let's play with composite variables.<br/>
When you need to collect a row with a set of attribute you can do it like this :

In [37]:
DO $$                      
DECLARE
rt_myrow categories%ROWTYPE;  
rec_myrow RECORD;  
BEGIN                     
   SELECT * into strict rt_myrow FROM categories order by category LIMIT 1 OFFSET 5 ; --STRICT raise an error if multi rows
   Raise info 'Variable with ROW TYPE : Id category : % - Category Name : % ',rt_myrow.category,rt_myrow.categoryname;                                                                                                               
   SELECT * into strict rec_myrow FROM categories order by category LIMIT 1 OFFSET 7 ; --STRICT raise an error if multi rows
   Raise info 'Variable with RECORD : Id category : % - Category Name : % ',rec_myrow.category,rec_myrow.categoryname;                                                                                                               

END                        
$$


INFO:  Variable with ROW TYPE : Id category : 6 - Category Name : Documentary 
INFO:  Variable with RECORD : Id category : 8 - Category Name : Family 


Are you ok with variables ?<br/>
From the preceding example, could you write an anonymous block to display the 11th row of the category table and get back the result into a composite variable with your own type ?<br/>


In [41]:
DROP TYPE if exists t_myType;
CREATE TYPE t_myType AS (category int, categoryname text) ;
                                                            
DO $$                      
DECLARE
myrow_type t_myType;                                                         
BEGIN                     
   SELECT category, categoryname FROM categories order by category limit 1 OFFSET 10  into strict myrow_type ;
   Raise info 'Id category : % - Category Name : % ',myrow_type.category,myrow_type.categoryname; 
                                                                                                                          
END                        
$$

INFO:  Id category : 11 - Category Name : Horror 


Now you are ready to developp in PL/SQL ;-).

 ### <span style="color:red">EXERCISES </span>

 ### <span style="color:blue">EX - 1 </span>
 
 Create an anonymous block computing the hypotenuse of a right-angled triangle where the opposite side sizes 5.5 and the adjacent side size 10.<br/>

All mathematique operators are <a href="https://docs.postgresql.fr/11/functions-math.html">here</a>

In [42]:
DO $$                      
DECLARE
a float4 default 5.5;
b a%TYPE = 10;
BEGIN                     
   Raise info 'hypotenuse %',|/(a^2 + b::float4^2); 
END                        
$$
LANGUAGE plpgsql;

INFO:  hypotenuse 11.4127122105133


 ### <span style="color:blue">EX - 2 </span>
 
 Transform the anonymous block to a function "hypo" accepting 2 arguments a and b.

In [43]:

CREATE or replace FUNCTION f_hypo(a float4, b float4) RETURNS float4 
AS $$
BEGIN                     
   return |/(a^2 + b::float4^2);
END                        
$$ 
LANGUAGE plpgsql;

In [44]:
select f_hypo(3.5,56.4);

1 row(s) returned.


f_hypo
56.5085


 ### <span style="color:blue">EX - 3 </span>

Guess the output of this anonymous procedure :

In [45]:
DO $$                      
<< level1 >>
DECLARE
v_level otypedef.otype_descr%TYPE = 'level1';
BEGIN 
   Raise info 'bloc1 -> level = %' ,v_level;  
   << level2_1 >>
   DECLARE
   v_level otypedef.otype_descr%TYPE = 'level2_1';
   BEGIN
       Raise info 'bloc2_1 -> level = % ',v_level; 
       Raise info 'bloc2_1 -> level1.level = % ',level1.v_level; 
       << level3_1 >>
       DECLARE
       v_level otypedef.otype_descr%TYPE = 'level3_1_1';
       BEGIN    
           Raise info 'bloc3_1_1 -> level = % ',v_level; 
           Raise info 'bloc3_1_1 -> level1.level = % ',level1.v_level;     
           Raise info 'bloc3_1_1 -> level2_1.level = % ',level2_1.v_level;  
           raise exception 'test';
        END level3_1;
        EXCEPTION
            WHEN others then
            Raise info 'Erreur id  % ',SQLSTATE; 
            Raise info 'Erreur detected  % ',SQLERRM; 
   END level2_1;
   << level2_2 >>
   DECLARE
   v_level otypedef.otype_descr%TYPE = 'level2_2';
   BEGIN
       Raise info 'bloc2_2 -> level = % ',v_level; 
       Raise info 'bloc2_2 -> level1.level = % ',level1.v_level; 
   END level2_2;
END level1 ;                     
$$ 


INFO:  bloc1 -> level = level1
INFO:  bloc2_1 -> level = level2_1 
INFO:  bloc2_1 -> level1.level = level1 
INFO:  bloc3_1_1 -> level = level3_1_1 
INFO:  bloc3_1_1 -> level1.level = level1 
INFO:  bloc3_1_1 -> level2_1.level = level2_1 
INFO:  Erreur id  P0001 
INFO:  Erreur detected  test 
INFO:  bloc2_2 -> level = level2_2 
INFO:  bloc2_2 -> level1.level = level1 


  ### <span style="color:blue">EX - 4</span>

In this exercice, you will create a course table as defined below :

In [53]:
drop table if exists courses;
CREATE TABLE  courses
(
  idcourse character varying(10),
  room character varying(100),
  teacher character varying(50),
  teacher_phone character varying(10)
);
insert into courses values ('CS101','Hall 20','George','0651482192');
insert into courses values ('CS154','Auditorium 01','Atkins','0651927291');
insert into courses values ('CS152','Hall 21','Atkins','0651927291');
insert into courses values ('CS102','Hall 21','George','0651482192');
select * from courses;

4 row(s) returned.


idcourse,room,teacher,teacher_phone
CS101,Hall 20,George,0651482192
CS154,Auditorium 01,Atkins,0651927291
CS152,Hall 21,Atkins,0651927291
CS102,Hall 21,George,0651482192


This relation is not very well designed, could you transform it in the Boyce-Codd normal form ?<br/>
Define and create all relations which will be used to store data in the new data model :

In [54]:
drop table if exists lecons;
drop table if exists rooms;
drop table if exists instructors;
CREATE TABLE rooms
(
  roomname character varying(10) NOT NULL,
  roomtype character varying(10),
  CONSTRAINT rooms_pkey PRIMARY KEY (roomname)
);

CREATE TABLE instructors
(
  teacher character varying(50) NOT NULL,
  phone character varying(10),
  CONSTRAINT instructors_pkey PRIMARY KEY (teacher)
);

drop table if exists lecons;
CREATE TABLE lecons
(
  idcourse character varying(10) NOT NULL,
  roomname character varying(100),
  teacher character varying(50),
  CONSTRAINT lecons_pkey PRIMARY KEY (idcourse),
  CONSTRAINT lecons_roomname_fkey FOREIGN KEY (roomname)
      REFERENCES rooms (roomname) MATCH SIMPLE
      ON UPDATE NO ACTION ON DELETE NO ACTION,
  CONSTRAINT lecons_teacher_fkey FOREIGN KEY (teacher)
      REFERENCES instructors (teacher) MATCH SIMPLE
      ON UPDATE NO ACTION ON DELETE NO ACTION
)


NOTICE:  la table « lecons » n'existe pas, poursuite du traitement


Now you are ready to migrate courses to your new design.

Create a stored procedure which will handle  data from the courses table and will translate to your new design schema.


In [55]:
DROP procedure if exists courseMigration();
create procedure courseMigration() 
AS
$BODY$
DECLARE
BEGIN 
    insert into instructors  select distinct teacher, teacher_phone from courses;
    insert into rooms select split_part(v_address, ' ',2) roomname, split_part(v_address, ' ',1) roomtype from (select distinct room  v_address from courses)t;
    insert into lecons select distinct idcourse,split_part(room, ' ',2) roomname, teacher from courses;
END;
$BODY$
LANGUAGE 'plpgsql';

In [56]:
call courseMigration();

Check all your data is well migrated

In [57]:
select * from lecons;

4 row(s) returned.


idcourse,roomname,teacher
CS152,21,Atkins
CS154,01,Atkins
CS101,20,George
CS102,21,George


In [58]:
select * from rooms;

3 row(s) returned.


roomname,roomtype
20,Hall
21,Hall
01,Auditorium


In [59]:
select * from instructors;

2 row(s) returned.


teacher,phone
George,0651482192
Atkins,0651927291


 ### <span style="color:blue">EX - 5 </span>

Guess the output of this anonymous block :

In [60]:
DO $$
DECLARE
    cur_ref refcursor;
    rec_row RECORD;
BEGIN
    OPEN cur_ref FOR SELECT * FROM categories  order by category;
    FETCH FIRST FROM cur_ref into rec_row;
    IF NOT FOUND THEN
        raise info 'No data found';
        raise exception 'No data found';
    END IF;
    FETCH cur_ref into rec_row;
    raise info 'Cursor position %', rec_row.category;
    MOVE NEXT FROM cur_ref;
    FETCH cur_ref INTO rec_row;
    raise info 'Cursor position %', rec_row.category;
    FETCH cur_ref INTO rec_row;
    raise info 'Cursor position %', rec_row.category;
    MOVE FORWARD 2 FROM cur_ref;
    FETCH cur_ref INTO rec_row;
    raise info 'Cursor position %', rec_row.category;
    
    MOVE LAST FROM cur_ref;
    FETCH cur_ref INTO rec_row;
    IF NOT FOUND THEN
        raise info 'Cursor out of result';
        MOVE RELATIVE -2 from cur_ref;
        FETCH cur_ref INTO rec_row;
    END IF;
    raise info 'Cursor position%', rec_row.category;
    CLOSE cur_ref;
END;
$$ 
;


INFO:  Cursor position 2
INFO:  Cursor position 4
INFO:  Cursor position 5
INFO:  Cursor position 8
INFO:  Cursor out of result
INFO:  Cursor position16


### <span style="color:blue">EX - 6 </span>
Create a function f_getWay accepting a customerid and returning street name of a customer.<br/>
Use the address1 column from the customers table.<br/>
You may use the string function split_part to help you.<br/>

All string functions are described <a href="https://www.postgresql.org/docs/current/functions-string.html">here</a>






In [61]:
DROP function if exists f_getWay(integer);
CREATE or replace FUNCTION f_getWay(custid integer) RETURNS  customers.address1%TYPE
AS $$
DECLARE
v_address customers.address1%TYPE; 
v_street customers.address1%TYPE;
BEGIN     
   EXECUTE 'select address1 from  customers where customerid = $1' into strict v_address using custid;
   raise info 'v_address % ',v_address;
   v_street = split_part(v_address, ' ',2);
   return v_street;
END                        
$$ 
LANGUAGE plpgsql;

NOTICE:  référence de type customers.address1%TYPE convertie en character varying


In [62]:
select f_getWay(1);

1 row(s) returned.
INFO:  v_address 4608499546 Dell Way 


f_getway
Dell


 ### <span style="color:blue">EX - 7 </span>
 Write a function "f_hi" taking one argument "username" and return either "Good morning" or "Good afternoon"   according to the current time (now).<br/>
Help : you could use the function now() to get back the current time and to_char() to get the hour.
 


In [63]:
CREATE OR REPLACE FUNCTION f_hi(v_username text) RETURNS text
AS $BODY$
DECLARE
v_label text;
v_htime integer;
BEGIN
    v_htime := to_char(now(), 'HH24')::int; 
    IF v_htime > 12 THEN
        v_label := 'Good afternoon'; 
    ELSE
        v_label := 'Good morning';
    END IF;
    RETURN v_label||' '||v_username||' !'; END
$BODY$
LANGUAGE plpgsql;

In [64]:
select f_hi('you');

1 row(s) returned.


f_hi
Good morning Hugo !


### <span style="color:blue">EX - 8 </span>


Create a function getExpensiveProduct which Reads the whole products table with a cursor and <br/>
displays with 'raise info' all products names more expensive than the input argument v_maxprice.
The function will return the number of products.

In [65]:
CREATE OR REPLACE FUNCTION getExpensiveProduct(v_maxprice float4) RETURNS integer 
AS
$BODY$
DECLARE
    -- cur_product refcursor; 
    row_resultat products%ROWTYPE; 
    v_index integer;
    cur_product CURSOR FOR select title,price from  products where price > v_maxprice;
BEGIN
    -- OPEN cur_product FOR SELECT * FROM products WHERE price > v_maxprice; 
    OPEN cur_product;
    LOOP
        FETCH cur_product INTO row_resultat; 
        IF NOT FOUND THEN
            EXIT; 
        END IF;
        RAISE NOTICE 'Product (%, %) greater than %)',row_resultat.title, row_resultat.price, v_maxprice;
    END LOOP; 
    v_index = cur_product.count; -- Cursors have some systems variables like count
    close cur_product;
    RETURN v_index;
END $BODY$
LANGUAGE 'plpgsql' VOLATILE;

In [66]:
select getExpensiveProduct(5);

1 row(s) returned.
NOTICE:  Product (BAKED VOYAGE, 25.99) greater than 5)
NOTICE:  Product (BAKED WAGON, 26.99) greater than 5)
NOTICE:  Product (BAKED WAIT, 24.99) greater than 5)
NOTICE:  Product (BAKED WAKE, 13.99) greater than 5)
NOTICE:  Product (BAKED WALLS, 22.99) greater than 5)
NOTICE:  Product (BAKED WANDA, 25.99) greater than 5)
NOTICE:  Product (BAKED WAR, 19.99) greater than 5)
NOTICE:  Product (BAKED WARDROBE, 11.99) greater than 5)
NOTICE:  Product (BAKED WARLOCK, 26.99) greater than 5)
NOTICE:  Product (BAKED WARS, 28.99) greater than 5)
NOTICE:  Product (BAKED WASH, 27.99) greater than 5)
NOTICE:  Product (BAKED WASTELAND, 25.99) greater than 5)
NOTICE:  Product (BAKED WATCH, 14.99) greater than 5)
NOTICE:  Product (BAKED WATERFRONT, 11.99) greater than 5)
NOTICE:  Product (BAKED WATERSHIP, 23.99) greater than 5)
NOTICE:  Product (BAKED WEDDING, 11.99) greater than 5)
NOTICE:  Product (BAKED WEEKEND, 17.99) greater than 5)
NOTICE:  Product (BAKED WEREWOLF, 27.99) great

getexpensiveproduct
49999


### <span style="color:blue">EX - 9 </span>
Write a function "f_inverse" which inverses the input value and returns the result.<br/>
Help  : use the WHILE control structure


In [67]:
drop function if exists f_inverse(varchar);
CREATE OR REPLACE FUNCTION f_inverse(varchar) RETURNS varchar
AS $PROC$
DECLARE
v_strin ALIAS FOR $1; 
v_strout varchar; 
v_strtemp varchar; 
v_intposition integer;
BEGIN
    -- Initialisation de str_out, sinon sa valeur reste à NULL
    v_strout := '';
    -- Suppression des espaces en début et fin de chaîne
    v_strtemp := trim(both ' ' from v_strin);
    -- position initialisée a la longueur de la chaîne
    -- la chaîne est traitée a l'envers
    v_intposition := char_length(v_strtemp);
    -- Boucle: Inverse l'ordre des caractères d'une chaîne de caractères 
    WHILE v_intposition > 0 LOOP
        -- la chaîne donnée en argument est parcourue
        -- à l'envers,
        -- et les caractères sont extraits individuellement au
        -- moyen de la fonction interne substring
        v_strout := v_strout || substring(v_strtemp, v_intposition, 1);
        v_intposition := v_intposition - 1; 
    END LOOP;
RETURN v_strout;
END; $PROC$
LANGUAGE plpgsql;

In [68]:
select f_inverse('toto');

1 row(s) returned.


f_inverse
otot


### <span style="color:blue">EX - 10 </span>

Write an anonymous block which counts in backwards from 10 to 0 by step 2

In [70]:
DO $$
DECLARE
v_counter int = 0;
BEGIN
    FOR v_counter IN REVERSE 10..1 BY 2  LOOP
      RAISE NOTICE 'Counter: %', v_counter;
    END LOOP;
END;
$$

NOTICE:  Counter: 10
NOTICE:  Counter: 8
NOTICE:  Counter: 6
NOTICE:  Counter: 4
NOTICE:  Counter: 2


Write a function which deletes all rows from the test table lower than i and return the number of deleted rows :<br/>
Help : try to do it with a parameterized cursor and delete rows from the cursor.

In [105]:
DROP function f_truncate(int);
CREATE FUNCTION f_truncate(int) RETURNS integer AS $$
DECLARE
    row_test RECORD;
    cur_test CURSOR (v_rowtodelete int) FOR SELECT * FROM test where i < v_rowtodelete;
    v_counter int = 0;
    v_i int = $1;
BEGIN
    OPEN cur_test(v_i);
    LOOP
         FETCH NEXT FROM cur_test into row_test;
         EXIT WHEN NOT FOUND;
         DELETE FROM test WHERE CURRENT OF cur_test;
         v_counter =v_counter +1;
         RAISE NOTICE 'row % deleted', row_test.i;
    END LOOP;
    close cur_test;
    RAISE NOTICE 'Clean up done';
    RETURN v_counter;
END;
$$ LANGUAGE plpgsql;

In [107]:
select f_truncate(5);

1 row(s) returned.
NOTICE:  row 3 deleted
NOTICE:  row 4 deleted
NOTICE:  Clean up done


f_truncate
2


Check your data is well cleaned up.

In [108]:
select * from test where i < 5;

0 row(s) returned.


### <span style="color:blue">EX - 11</span>

Modify the following program in order to catch the exception and raise a warning message describing the error encountered.

In [ ]:
DO $$                      
DECLARE
myrow_rowtype categories%ROWTYPE;                                                         
BEGIN                     
   SELECT * FROM categories into strict myrow_rowtype ;
   Raise info 'Id category : % - Category Name : % ',myrow_rowtype.category,myrow_rowtype.categoryname; 
EXCEPTION
    WHEN too_many_rows then
    Raise warning 'Catch an error %',SQLSTATE;
    Raise warning 'Description : %',SQLERRM;
    WHEN others then
    Raise warning 'Catch an other error %',SQLSTATE;
    Raise warning 'Description : %',SQLERRM; 
END                        
$$

Write a division function where inputs are 2 floats and return the  result of the division


In [109]:
CREATE OR REPLACE FUNCTION division(v_arg1 float, v_arg2 float) RETURNS float4
AS $BODY$
BEGIN
    RETURN v_arg1::float4/v_arg2::float4; END
$BODY$
LANGUAGE plpgsql;



In [110]:
select division(6,7)

1 row(s) returned.


division
0.857143


What happens if you set up the second argument to 0

In [111]:
select division(6,0)

1 row(s) returned.
LOG:  Watch out, [22012]: division par zéro


division
nan


Catch the exception and return the value Nan :

In [112]:
CREATE OR REPLACE FUNCTION division(arg1 integer, arg2 integer) RETURNS float4 AS
$BODY$
BEGIN
    RETURN arg1::float4/arg2::float4; 
    EXCEPTION WHEN OTHERS THEN
    -- attention, division par zéro
    RAISE LOG 'Watch out, [%]: %', SQLSTATE, SQLERRM;
    RETURN 'NaN'; 
END $BODY$
LANGUAGE 'plpgsql' VOLATILE;

In [113]:
select division(6,0)

1 row(s) returned.
LOG:  Watch out, [22012]: division par zéro


division
nan


### <span style="color:blue">EX - 12 </span>

In this exercice, we will trace all modifications(delete, insert, update ) run on the products tables into an audit table products_audit.
In the products_audit table, we will save all old values in product before modification.
Help : you could use the following DDL to create the products_audit table.


In [115]:
DROP TABLE IF EXISTS products_audit;
CREATE TABLE products_audit
(
    operation         char(1)   NOT NULL,
    stamp             timestamp NOT NULL,
    userid            text      NOT NULL,
    prod_id serial NOT NULL,
    category smallint NOT NULL,
    title text NOT NULL,
    actor text NOT NULL,
    price numeric NOT NULL,
    special smallint,
    common_prod_id integer NOT NULL
)

Create the "f_products_audit" function which will save all old values 

In [117]:
DROP TRIGGER IF EXISTS tr_products_audit on products;
DROP FUNCTION IF EXISTS f_products_audit();
CREATE OR REPLACE FUNCTION f_products_audit() RETURNS TRIGGER
AS $BODY$
    BEGIN
        --
        -- Create a row in products_audit to reflect the operation performed on products,
        -- make use of the special variable TG_OP to work out the operation.
        --
        IF (TG_OP = 'DELETE') THEN
            INSERT INTO products_audit SELECT 'D', now(), user, OLD.*;
        ELSIF (TG_OP = 'UPDATE') THEN
            INSERT INTO products_audit SELECT 'U', now(), user, NEW.*;
        ELSIF (TG_OP = 'INSERT') THEN
            INSERT INTO products_audit SELECT 'I', now(), user, NEW.*;
        END IF;
        RETURN NULL; -- result is ignored since this is an BEFORE trigger
    END;
$BODY$
LANGUAGE 'plpgsql' ;


Create a trigger "tr_products_audit" firing the  f_products_audit before commiting modifications.

In [118]:
DROP TRIGGER IF EXISTS tr_products_audit on products;
CREATE TRIGGER tr_products_audit
BEFORE INSERT OR UPDATE OR DELETE ON products
    FOR EACH ROW EXECUTE FUNCTION f_products_audit();


NOTICE:  le trigger « tr_products_audit » de la relation « products » n'existe pas, poursuite du traitement


Try to update/ insert a new product :

In [119]:
select title,price from products where title ='ACADEMY BEAR';

1 row(s) returned.


title,price
ACADEMY BEAR,0.143904


In [122]:
update products set price= price * 1.2 where title ='ACADEMY BEAR';

Check your products_audit table :

In [123]:
select now(),* from products_audit;

2 row(s) returned.


now,operation,stamp,userid,prod_id,category,title,actor,price,special,common_prod_id
2019-01-23 08:19:44.227392+01:00,U,2019-01-23 08:19:16.047849,ds2,59,7,ACADEMY BEAR,LAUREN WASHINGTON,0.172685,0,38089
2019-01-23 08:19:44.227392+01:00,U,2019-01-23 08:19:41.593872,ds2,59,7,ACADEMY BEAR,LAUREN WASHINGTON,0.172685,0,38089


### <span style="color:blue">EX - 13 </span>

The manager wants to see all orders per customer.<br/>
For this exercice, we will limit orders to the customer where customerid is 11769.<br/>
help: you will need to access to the customers and orders table.

Create a view 'view_custOrders' displaying all orders attached to the customerid 11769 ?<br/>
Note that the view should contain following informations : customerid,firstname,lastname,orderdate,tax,orderid 


In [124]:
drop view if exists view_custOrders;
create or replace view view_custOrders  as select C.customerid,firstname,lastname,orderdate,tax,orderid 
from customers C INNER join orders O on C.customerid=O.customerid 
where C.customerid =11769;

Check the content of the view 

In [125]:
select * from view_custOrders;

5 row(s) returned.


customerid,firstname,lastname,orderdate,tax,orderid
11769,WDQWVU,EHXFHFISJU,2009-06-10,20,25278
11769,WDQWVU,EHXFHFISJU,2009-08-25,15.09,37795
11769,WDQWVU,EHXFHFISJU,2009-09-08,20.33,43350
11769,WDQWVU,EHXFHFISJU,2009-12-29,8.1,55074
11769,WDQWVU,EHXFHFISJU,2009-12-31,21.83,57277


From the view_custOrders view, the manager must be able to delete and update the customer orders.

Create a function update_custOrder_view which will allow  :
 * to delete any order
 * to update following fields : firstname, lastname, orderdate,tax


In [126]:
CREATE OR REPLACE FUNCTION update_custOrder_view() RETURNS TRIGGER AS $$
    BEGIN
        -- The TG_OP variable give the event name  which will fire the trigger
        IF (TG_OP = 'DELETE') THEN
            -- if we delete an order we need to delete all products attached to the order
            DELETE FROM orderlines WHERE orderid = OLD.orderid;
            DELETE FROM orders WHERE orderid = OLD.orderid;
            IF NOT FOUND THEN RETURN NULL; END IF;
        ELSIF (TG_OP = 'UPDATE') THEN
            UPDATE orders
            SET 
            orderdate = NEW.orderdate,tax = NEW.tax WHERE orderid = OLD.orderid;
            UPDATE customers 
            SET 
            firstname = NEW.firstname,lastname= NEW.lastname  WHERE customerid = OLD.customerid; 
            IF NOT FOUND THEN RETURN NULL; END IF;
        END IF;
        RETURN NULL;
    END;
$$ LANGUAGE plpgsql;

Create the trigger that will fire the update_custOrder_view function.

In [127]:
DROP TRIGGER IF EXISTS tr_update_custOrder_view on view_custOrders;
CREATE TRIGGER tr_update_custOrder_view
INSTEAD OF UPDATE OR DELETE ON view_custOrders
FOR EACH ROW EXECUTE FUNCTION update_custOrder_view();

NOTICE:  le trigger « tr_update_custorder_view » de la relation « view_custorders » n'existe pas, poursuite du traitement


Delete the order (21499)

In [ ]:
delete from view_custOrders where orderid=21499;

Check the order (21499) is no more available 

In [ ]:
select * from view_custOrders;

Check the order is well deleted from ordelines and orders tables :

In [ ]:
select * from orderlines where orderid=21499;

In [ ]:
select * from orders where orderid=21499;

Now increase the tax to 20 for the order (37795)

In [ ]:
update view_custOrders set tax=20 where orderid=25278;

Check the view and tables are well updated  

In [ ]:
select * from view_custOrders where orderid=25278;

In [ ]:
select * from orders where orderid=25278;

### <span style="color:blue">EX - 14 </span>

Write a py_max function returning the maximum value between 2 arguments

In [129]:
DROP FUNCTION py_max;
CREATE FUNCTION py_max (a integer, b integer)
  RETURNS integer
AS $$
  if a > b:
    return a
  return b
$$ LANGUAGE plpython2u;

In [130]:
select pymax(2,4)

1 row(s) returned.


pymax
4
